In [1]:
from datasets import load_dataset

In [2]:
# Model
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# Prompts
ds = load_dataset("lighteval/pile_helm", "arxiv")["test"]
ds = ds.select(range(5))
prompts = [sample["text"] for sample in ds]

backend = "pytorch"

use_optimum = False


# PyTorch


In [3]:
import os
from pytorth_experiment_fns import *
from accelerate import notebook_launcher

# Restrict to GPUs 0 and 1
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

notebook_launcher(
    lambda: run_experiment(
        model_name=model_name,
        prompts=prompts,
        inference_fn=run_gen_inference_with_metrics,
        task_type="text_generation",
        backend="pytorch",
        use_optimum=False,
        max_input_tokens=512,
        max_output_tokens=50,
        batch_size=8
    ),
    num_processes=2
)


Launching training on 2 GPUs.
Using device: cuda:0 (Local Rank: 0)
Model is on cuda:1Using 2 GPUs: [0, 1]

Model is on cuda:0

[codecarbon WARNING @ 19:32:58] Multiple instances of codecarbon are allowed to run at the same time.


[codecarbon INFO @ 19:32:58] [setup] RAM Tracking...
[codecarbon WARNING @ 19:32:58] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 19:32:58] [setup] CPU Tracking...
[codecarbon WARNING @ 19:32:58] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU

[codecarbon INFO @ 19:32:58] [setup] RAM Tracking...
[codecarbon INFO @ 19:32:58] [setup] CPU Tracking...
[codecarbon WARNING @ 19:32:58] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU

[codecarbon INFO @ 19:32:59] CPU Model on constant consumption mode: AMD EPYC 7742 64-Core Processor
[codecarbon INFO @ 19:32:59] [setup] GPU Tracking...
[codecarbon INFO @ 19:32:59] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 19:32:59] CPU Model on constant consumption mode: AMD EPYC 

AMD CPU detected. Using AMD energy monitoring (or custom estimation).


W0304 19:33:08.179000 3220394 torch/multiprocessing/spawn.py:160] Terminating process 3220576 via signal SIGTERM
E0304 19:33:08.636000 3220394 torch/distributed/elastic/multiprocessing/api.py:732] failed (exitcode: 1) local_rank: 0 (pid: 3220575) of fn: <lambda> (start_method: fork)
E0304 19:33:08.636000 3220394 torch/distributed/elastic/multiprocessing/api.py:732] Traceback (most recent call last):
E0304 19:33:08.636000 3220394 torch/distributed/elastic/multiprocessing/api.py:732]   File "/home/228755@hertie-school.lan/thesis/thesis/lib/python3.10/site-packages/torch/distributed/elastic/multiprocessing/api.py", line 687, in _poll
E0304 19:33:08.636000 3220394 torch/distributed/elastic/multiprocessing/api.py:732]     self._pc.join(-1)
E0304 19:33:08.636000 3220394 torch/distributed/elastic/multiprocessing/api.py:732]   File "/home/228755@hertie-school.lan/thesis/thesis/lib/python3.10/site-packages/torch/multiprocessing/spawn.py", line 203, in join
E0304 19:33:08.636000 3220394 torch/di

ChildFailedError: 
============================================================
<lambda> FAILED
------------------------------------------------------------
Failures:
  <NO_OTHER_FAILURES>
------------------------------------------------------------
Root Cause (first observed failure):
[0]:
  time      : 2025-03-04_19:33:07
  host      : ds01
  rank      : 0 (local_rank: 0)
  exitcode  : 1 (pid: 3220575)
  error_file: /tmp/torchelastic_s_f36qmm/none_ikoa4wxx/attempt_0/0/error.json
  traceback : Traceback (most recent call last):
    File "/home/228755@hertie-school.lan/thesis/thesis/lib/python3.10/site-packages/torch/distributed/elastic/multiprocessing/errors/__init__.py", line 355, in wrapper
      return f(*args, **kwargs)
    File "/tmp/ipykernel_3220394/94175995.py", line 9, in <lambda>
      lambda: run_experiment(
    File "/home/228755@hertie-school.lan/thesis/pytorth_experiment_fns.py", line 428, in run_experiment
      codecarbon_data = stop_energy_tracking(tracker)
    File "

# vLLM

NB: memory problems with vLLM

In [ ]:
from vllm_experiment import *

In [ ]:
results = run_experiment_vllm(
    model_name=model_name,
    prompts=prompts,
    task_type="text_generation",
    max_input_tokens=512,
    max_output_tokens=50,
    batch_size=8
)

print(results)

In [ ]:
from optimum_benchmark import Benchmark, BenchmarkConfig, TorchrunConfig, InferenceConfig, PyTorchConfig
from optimum_benchmark.logging_utils import setup_logging

setup_logging(level="INFO", handlers=["console"])

if __name__ == "__main__":
    launcher_config = TorchrunConfig(nproc_per_node=2)
    scenario_config = InferenceConfig(latency=True, memory=True)
    backend_config = PyTorchConfig(model="gpt2", device="cuda", device_ids="0,1", no_weights=True)
    benchmark_config = BenchmarkConfig(
        name="pytorch_gpt2",
        scenario=scenario_config,
        launcher=launcher_config,
        backend=backend_config,
    )
    benchmark_report = Benchmark.launch(benchmark_config)

    # convert artifacts to a dictionary or dataframe
    benchmark_config.to_dict() # or benchmark_config.to_dataframe()

    # save artifacts to disk as json or csv files
    benchmark_report.save_csv("benchmark_report.csv") # or benchmark_report.save_json("benchmark_report.json")

    # push artifacts to the hub
    benchmark_config.push_to_hub("IlyasMoutawwakil/pytorch_gpt2") # or benchmark_config.push_to_hub("IlyasMoutawwakil/pytorch_gpt2")

    # or merge them into a single artifact
    benchmark = Benchmark(config=benchmark_config, report=benchmark_report)
    benchmark.save_json("benchmark.json") # or benchmark.save_csv("benchmark.csv")
    benchmark.push_to_hub("IlyasMoutawwakil/pytorch_gpt2")

    # load artifacts from the hub
    benchmark = Benchmark.from_hub("IlyasMoutawwakil/pytorch_gpt2") # or Benchmark.from_hub("IlyasMoutawwakil/pytorch_gpt2")

    # or load them from disk
    benchmark = Benchmark.load_json("benchmark.json") # or Benchmark.load_csv("benchmark_report.csv")